In [ ]:
!pip install langchain boto3
!pip install transformers
!pip install sentence_transformers

In [ ]:
pip install langchain==0.0.224

In [ ]:
pip install boto3 --upgrade

In [ ]:
!pip install ~/SageMaker/botocore-1.29.162-py3-none-any.whl
!pip install ~/SageMaker/boto3-1.26.162-py3-none-any.whl

In [ ]:
import boto3
from langchain.retrievers import AmazonKendraRetriever
from typing import List
from typing import Dict
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
#from langchain import SagemakerEndpoint, LLMChain
from langchain import LLMChain
from langchain.llms.bedrock import Bedrock
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json

from langchain.docstore.document import Document
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [ ]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generated_text"]



content_handler = ContentHandler()


In [49]:
from langchain import PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

#llm2 = Bedrock(
#model_id="anthropic.claude-v1",
#model_kwargs={'max_tokens_to_sample': 300})

maxTokensToSample = 300
temp = 0.6
topP = 0.9


#modelArgs = {'maxTokens': int(maxTokensToSample), 'temperature':float(temp), "topP": float(topP),"stop_sequences":[]}
modelArgs = {'maxTokens': int(maxTokensToSample) , 'temperature':float(temp), "topP": float(topP)}

llm2 = Bedrock(
model_id="ai21.j2-jumbo-instruct",
model_kwargs=modelArgs)

#llm2.debug = True

In [50]:
llm_query= "my airpods are not connecting to the iphone, how do I fix that "

prompt_template = """
  {context}
    {question}
  """
  
PROMPT = PromptTemplate(
      template=prompt_template, input_variables=["context", "question"]
  )
  
chain = load_qa_chain(llm=llm2, prompt=PROMPT, verbose=True)

  

In [51]:
output = chain({"input_documents":"", "question": llm_query}, return_only_outputs=False)
print(output['output_text'])




> Entering new  chain...


> Entering new  chain...
Prompt after formatting:

  
    my airpods are not connecting to the iphone, how do I fix that 
  


ValueError: Error raised by bedrock service: An error occurred (ValidationException) when calling the InvokeModel operation: The provided inference configurations are invalid